# Association Rules

## Group Members
1. Bett Kipkemoi
2. Faith Ngina
3. Dominic Ong'aro
4. Dennis Mmenya Mwange

## Implementing Apriori, FP-Growth and Eclat
Use both Apriori, FP-Growth and Eclat to analyze a dataset of retail transactions in Python. Compare their performance in terms of execution time and memory usage.

In [18]:
# import libraries
import time
import pandas as pd
from itertools import combinations
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

In [19]:
# read the file
def read_transactions(path):
	"""Read the CSV where each row is a comma-separated transaction."""
	transactions = []
	with open(path, 'r', encoding='utf-8') as f:
		for line in f:
			parts = [p.strip() for p in line.strip().split(',') if p.strip()]
			if parts:
				transactions.append(parts)
	return transactions

In [20]:
## eclat
def eclat_bruteforce(transactions, min_support=0.05):
	"""Simple Eclat-style frequent itemset discovery by counting combinations.
	This is a brute-force implementation suitable for small item counts.
	Returns a DataFrame with columns ['support', 'itemsets'].
	"""
	n = len(transactions)
	# unique items
	items = sorted({item for t in transactions for item in t})
	results = []
	max_k = len(items)
	for k in range(1, max_k + 1):
		any_found = False
		for combo in combinations(items, k):
			count = 0
			combo_set = set(combo)
			for t in transactions:
				if combo_set.issubset(t):
					count += 1
			support = count / n
			if support >= min_support:
				any_found = True
				results.append({'support': support, 'itemsets': frozenset(combo)})
		if not any_found:
			# no frequent itemsets of this size -> larger sizes will not be frequent
			break
	if results:
		df = pd.DataFrame(results).sort_values(by=['support', 'itemsets'], ascending=[False, True])
	else:
		df = pd.DataFrame(columns=['support', 'itemsets'])
	return df

In [21]:
## main function
def main():
	data_path = 'Breakfast_Transactions.csv'
	print('Reading transactions from', data_path)
	transactions = read_transactions(data_path)
	print(f'Number of transactions: {len(transactions)}')

	# One-hot encode
	te = TransactionEncoder()
	te_ary = te.fit(transactions).transform(transactions)
	df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

	min_support = 0.05
	min_confidence = 0.6

	# Apriori
	t0 = time.perf_counter()
	fi_apriori = apriori(df_onehot, min_support=min_support, use_colnames=True)
	t1 = time.perf_counter()
	apriori_time = t1 - t0
	print('\nApriori found', len(fi_apriori), 'itemsets in', f'{apriori_time:.3f}s')
	fi_apriori.to_csv('frequent_itemsets_apriori.csv', index=False)

	# FP-Growth
	t0 = time.perf_counter()
	fi_fpgrowth = fpgrowth(df_onehot, min_support=min_support, use_colnames=True)
	t1 = time.perf_counter()
	fpgrowth_time = t1 - t0
	print('FPGrowth found', len(fi_fpgrowth), 'itemsets in', f'{fpgrowth_time:.3f}s')
	fi_fpgrowth.to_csv('frequent_itemsets_fpgrowth.csv', index=False)

	# Eclat (brute-force)
	t0 = time.perf_counter()
	fi_eclat = eclat_bruteforce(transactions, min_support=min_support)
	t1 = time.perf_counter()
	eclat_time = t1 - t0
	print('Eclat (bruteforce) found', len(fi_eclat), 'itemsets in', f'{eclat_time:.3f}s')
	fi_eclat.to_csv('frequent_itemsets_eclat.csv', index=False)

	# Generate association rules for apriori results
	rules_apriori = association_rules(fi_apriori, metric='confidence', min_threshold=min_confidence)
	rules_fpgrowth = association_rules(fi_fpgrowth, metric='confidence', min_threshold=min_confidence)
	# For eclat, association_rules expects a DataFrame with itemsets in 'itemsets' column; our df matches that
	rules_eclat = association_rules(fi_eclat.rename(columns={'itemsets': 'itemsets', 'support': 'support'}),
									metric='confidence', min_threshold=min_confidence) if not fi_eclat.empty else pd.DataFrame()

	print('\nRules found:')
	print('Apriori rules:', len(rules_apriori))
	print('FPGrowth rules:', len(rules_fpgrowth))
	print('Eclat rules:', len(rules_eclat))

	# Save rules
	rules_apriori.to_csv('rules_apriori.csv', index=False)
	rules_fpgrowth.to_csv('rules_fpgrowth.csv', index=False)
	rules_eclat.to_csv('rules_eclat.csv', index=False)

	# Summary
	print('\nSummary:')
	print(f'Apriori: {len(fi_apriori)} itemsets, {len(rules_apriori)} rules, time={apriori_time:.3f}s')
	print(f'FPGrowth: {len(fi_fpgrowth)} itemsets, {len(rules_fpgrowth)} rules, time={fpgrowth_time:.3f}s')
	print(f'Eclat: {len(fi_eclat)} itemsets, {len(rules_eclat)} rules, time={eclat_time:.3f}s')


if __name__ == '__main__':
	main()

Reading transactions from Breakfast_Transactions.csv
Number of transactions: 605

Apriori found 50 itemsets in 0.015s
FPGrowth found 50 itemsets in 0.005s
Eclat (bruteforce) found 50 itemsets in 0.047s

Rules found:
Apriori rules: 12
FPGrowth rules: 12
Eclat rules: 12

Summary:
Apriori: 50 itemsets, 12 rules, time=0.015s
FPGrowth: 50 itemsets, 12 rules, time=0.005s
Eclat: 50 itemsets, 12 rules, time=0.047s


## Performance Comparison of Association Rule Mining Algorithms

On the Breakfast_Transactions.csv dataset (605 transactions, min_support yielding 50 frequent itemsets and 12 rules):

### Execution Time (for finding frequent itemsets):

- FPGrowth: Fastest at 0.005 seconds — 3x faster than Apriori and ~9x faster than Eclat.
- Apriori: Moderate at 0.015 seconds — efficient but slower due to multiple dataset scans and candidate generation.
- Eclat (bruteforce): Slowest at 0.047 seconds — significantly higher due to vertical data format processing and intersection operations in brute-force mode.

### Memory Usage (qualitative, based on typical behavior):
- FPGrowth: Generally most memory-efficient; compresses data into a compact FP-tree structure, ideal for dense or moderate-sized datasets like this.
- Apriori: Moderate memory use; stores candidate itemsets, which can grow exponentially with lower support (not an issue here with only 50 itemsets).
- Eclat: Higher memory consumption; maintains vertical tidsets (transaction ID lists) for each item, which can be memory-intensive even for small datasets.

## Conclusion: 
For this small dataset, FPGrowth clearly outperforms both Apriori and Eclat in execution time and likely memory efficiency, making it the best choice. Apriori offers a good balance, while brute-force Eclat is least efficient here. On larger or denser datasets, FPGrowth's advantages become even more pronounced.